In [ ]:
# Importo las librerias que voy a necesitar durante el análisis

import pandas as pd
import numpy as np

import xlrd
from datetime import datetime, date, timedelta

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import squarify


In [ ]:
# A partir de los 3 datasets que voy a usar para el análisis, genero por cada uno un data frame

df_1 = pd.read_csv('datasets/precert_ene_feb_2020.csv', encoding='latin-1')

df_2 = pd.read_csv('datasets/precert_mar_abr_2020.csv', encoding='latin-1', 
                   low_memory=False)

df_3 = pd.read_csv('datasets/precert_may_jun_2020.csv', encoding='latin-1')


In [ ]:
# Genero un data frame auxiliar para que no me duplique los indices

df = pd.DataFrame()

In [ ]:
# Concateno los 3 data frames en uno

df_cert_total = pd.concat([df, df_1, df_2, df_3])

In [ ]:
# Realizo una primer exploración de datos con el método head

df_cert_total.head()

In [ ]:
# Aplico el atributo columns para conocere los nombres de las columnas de mi df

df_cert_total.columns

In [ ]:
# Con el método drop elimino las columnas de datos que no aportan valor a este análsis

df_cert_total= df_cert_total.drop(['Tipo Doc.', 'País Doc.','Mutualista', 
                                   'Origen Certificación', 'Tipo Certificación', 
                                   'Tipo Control', 'Fecha Acto Médico', 
                                   'Fecha Desde Precertificación', 
                                   'Fecha Hasta Precertificación', 'Vig. Desde', 
                                   'Vig. Hasta','Demora Ingreso', 'Internación', 
                                   'Cód. Mutualista Internación',
                                   'Mutualista Internación',
                                   'Oficina Ingreso Precertificación','Localidad', 
                                   'Mutualista FONASA'], axis=1)

In [ ]:
# Renombro las columnas utilizando Sanke_case con el método rename

df_cert_total = df_cert_total.rename(columns={'N° Doc. ': 'documento', 
                                              'Sexo': 'sexo', 
                                              'Fecha Nac.': 'fecha_nacimiento', 
                                              'Fecha Ingreso Sistema': 'fecha_ingreso', 
                                              'Cantidad de días Precertif.': 'cantidad_días', 
                                              'Cód. Patología': 'código_pat', 
                                              'Patología': 'patología', 
                                              'Departamento': 'departamento'})

In [ ]:
# Verifico que el df tiene las columnas y nombres que necesito

df_cert_total.head()

In [ ]:
# Con el método info analizo los datos y tipo de datos que contiene el df

df_cert_total.info()

In [ ]:
# Aplicando el método drop_duplicates elimino los registros de certificación duplicados con los criterios indicados

df_cert_total = df_cert_total.drop_duplicates(subset=['documento', 'fecha_ingreso', 
                                                      'código_pat', 'cantidad_días'], 
                                              keep='last')

In [ ]:
# Con el atributo shape verifico si el tamaño del df se modifico luego del dropeo

df_cert_total.shape

In [ ]:
# Defino la función para normalizar las fechas

def norm_fecha(xl_date):
    return xlrd.xldate_as_datetime(xl_date, 0).date()


In [ ]:
# Aplico la función a Fecha Nacimiento y a Fecha Ingreso

df_cert_total.fecha_nacimiento = df_cert_total.fecha_nacimiento.apply(norm_fecha)

df_cert_total.fecha_ingreso = df_cert_total.fecha_ingreso.apply(norm_fecha)

print(df_cert_total)

In [ ]:
# Defino la funcion para obtener el dato mes a partir de la fecha

def fecha_mes(date_norm):
    return date_norm.month

In [ ]:
# Aplico la función para quedarme con el dato mes de Fecha Ingreso y lo agrego como columna al df

df_cert_total.mes_ingreso = df_cert_total.fecha_ingreso.apply(fecha_mes)

df_cert_total = df_cert_total.assign(mes_ingreso = df_cert_total.mes_ingreso)

print(df_cert_total)



In [ ]:
# Con el método value_counts cuento la cantidad de registros de certificación por mes y guardo la data en un diccionario

df_cantidad_cert_mes = (df_cert_total.mes_ingreso.value_counts())

#print(cantidad_cert_mes)

dict_cert_total = df_cantidad_cert_mes.to_dict()

print(dict_cert_total)

In [ ]:
# Genero un nuevo data frame con los registros de certificación con patologías que requiero para el análisis utilizando el método isin

df_cert_analisis = df_cert_total[df_cert_total.código_pat.isin(['J111', 'J22', 'J069', 'J989', 'J00', 'J209'])] 

print(df_cert_analisis)

In [ ]:
# Defino una función para calcular la edad de la persona a partir de la Fecha Nacimiento

def calculo_edad(fecha_nac):
    today = datetime.today()
    years = today.year-fecha_nac.year
    
    if today.month < fecha_nac.month or (today.month == fecha_nac.month and today.day < fecha_nac.day):
        years-=1
    
    return (years)


In [ ]:
# Aplico la función para obtener la edad y agrego el dato como nueva columna del df

df_cert_analisis.edad = df_cert_analisis.fecha_nacimiento.apply(calculo_edad)

#print(df_cert_analisis.edad)

df_cert_analisis = df_cert_analisis.assign(edad = df_cert_analisis.edad)

print(df_cert_analisis)



In [ ]:
# Con el método value_counts cuento la cantidad de registros de certificación por mes y guardo la data en un diccionario

df_cantidad_cert_J_mes = (df_cert_analisis.mes_ingreso.value_counts())


dict_cert_J = df_cantidad_cert_J_mes.to_dict()
print(dict_cert_J)

In [ ]:
# A partir de los diccionarios con los totales por mes, genero un data frame auxiliar

df_cert_aux = pd.DataFrame()
df_cert_aux = df_cert_aux.append(dict_cert_total, ignore_index = True)
df_cert_aux = df_cert_aux.append(dict_cert_J, ignore_index = True)
df_cert_aux

In [ ]:
# Cambio el nombre de los índices, traspongo los datos (columnas e índices) y órdeno los índices

df_cert_aux.index = ['Total', 'Cert_J']

df_cert_aux = df_cert_aux.transpose()

df_cert_aux = df_cert_aux.sort_index()

df_cert_aux

In [ ]:

def calculo_porcentual(total, J):
    return (J*100)/total


df_cert_aux.porcentaje = calculo_porcentual(df_cert_aux.Total, df_cert_aux.Cert_J)
df_cert_aux = df_cert_aux.assign(porcentaje = df_cert_aux.porcentaje)

df_cert_aux

In [ ]:
df_cert_total

In [ ]:
# A partir del data frame auxiliar se plantea la primer gráfica

n = len(df_cert_aux.index)
x = np.arange(n)
width = 0.35

plt.bar(x - width, df_cert_aux.Total, width=width, label='Certificaciones Totales', color=['cornflowerblue'])
plt.bar(x , df_cert_aux.Cert_J, width=width, label='Certificaciones pat_J', color=['midnightblue'])
plt.xticks(x, df_cert_aux.index)
plt.legend(loc='best')
plt.show


In [ ]:
# Defino un nuevo data frame a partir de agrupar y contar totales de Mes Ingreso del data frame del análisis

df_totales_J_mes = df_cert_analisis.groupby(['mes_ingreso']).size().reset_index(name='Count')

print(df_totales_J_mes)

In [ ]:
# A partir del data frame anterior se presenta la siguiente gráfica de tendencia

df_totales_J_mes.plot(x="mes_ingreso", y="Count", kind="line")
plt.show()

In [ ]:
# Defino un nuevo data frame a partir de agrupar y contar totales de Mes Ingreso y sexo del data frame del análisis

df_sexo = df_cert_analisis.groupby(['mes_ingreso', 'sexo']).size().reset_index(name='Count')

print(df_sexo)

In [ ]:
df_cuenta_sexo = (df_cert_analisis.sexo.value_counts())
df_cuenta_sexo

In [ ]:
# A partir del data frame anterior se grafica comparando cantidad de certificaciones por mes con distinción del genero

plt.figure(figsize=(8, 6))
sb.pointplot(x='mes_ingreso', 
              y='Count',
              hue='sexo', 
              palette='Blues',  
              data=df_sexo)
plt.xlabel("Mes ingreso")
plt.ylabel("Cantidad de certificaciones")
plt.title("Total por mes por genero")
plt.show()

In [ ]:
# Defino un nuevo data frame a partir de agrupar y contar totales de Mes Ingreso y edad del data frame del análisis

df_edad= df_cert_analisis.groupby(['mes_ingreso', 'edad']).size().reset_index(name='Count')

print(df_edad)

In [ ]:
#df_edad.min()
#df_edad.max()
df_edad.mean()

In [ ]:
# A partir del data frame de edad realizo un gráfico de dispersión

y = df_cert_analisis.mes_ingreso
x = df_cert_analisis.edad

data = df_cert_analisis

sb.scatterplot(data=data, x="mes_ingreso", y="edad")


plt.show()

In [ ]:
# Genero un nuevo data frame a partir de agrupar y contar totales por departamento del data frame del análisis, quito los valores 'sin departamento' con el método not isin

df_dpto= df_cert_analisis.groupby(['departamento']).size().reset_index(name='Count')


df_dpto = df_dpto[~df_dpto.departamento.isin(['Sin departamento'])] 

print(df_dpto)


In [ ]:
# Con la información del data frame de departamentos realizo gráfico del tipo treemap

fig, ax = plt.subplots(1, figsize = (15,15))
ax = squarify.plot(sizes=df_dpto.Count, label=df_dpto.departamento, 
                   alpha=0.5,color=sb.color_palette("Paired", len(df_dpto.departamento)))
plt.axis('off')
plt.show()